<a href="https://colab.research.google.com/github/devopsrudr/Breast-Cancer-Survival-Prediction/blob/main/Breast_Cancer_Survival_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
import plotly.express as px

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [4]:
df = pd.read_csv("/content/BRCA.csv")

In [6]:
print(df.head())

     Patient_ID   Age  Gender  Protein1  Protein2  Protein3  Protein4  \
0  TCGA-D8-A1XD  36.0  FEMALE  0.080353   0.42638   0.54715  0.273680   
1  TCGA-EW-A1OX  43.0  FEMALE -0.420320   0.57807   0.61447 -0.031505   
2  TCGA-A8-A079  69.0  FEMALE  0.213980   1.31140  -0.32747 -0.234260   
3  TCGA-D8-A1XR  56.0  FEMALE  0.345090  -0.21147  -0.19304  0.124270   
4  TCGA-BH-A0BF  56.0  FEMALE  0.221550   1.90680   0.52045 -0.311990   

  Tumour_Stage                      Histology ER status PR status HER2 status  \
0          III  Infiltrating Ductal Carcinoma  Positive  Positive    Negative   
1           II             Mucinous Carcinoma  Positive  Positive    Negative   
2          III  Infiltrating Ductal Carcinoma  Positive  Positive    Negative   
3           II  Infiltrating Ductal Carcinoma  Positive  Positive    Negative   
4           II  Infiltrating Ductal Carcinoma  Positive  Positive    Negative   

                  Surgery_type Date_of_Surgery Date_of_Last_Visit  \
0  Mo

In [8]:
print(df.isnull().sum())

Patient_ID             7
Age                    7
Gender                 7
Protein1               7
Protein2               7
Protein3               7
Protein4               7
Tumour_Stage           7
Histology              7
ER status              7
PR status              7
HER2 status            7
Surgery_type           7
Date_of_Surgery        7
Date_of_Last_Visit    24
Patient_Status        20
dtype: int64


this dataset has some null value,so we have to drop these nullvalues

In [9]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 317 entries, 0 to 333
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Patient_ID          317 non-null    object 
 1   Age                 317 non-null    float64
 2   Gender              317 non-null    object 
 3   Protein1            317 non-null    float64
 4   Protein2            317 non-null    float64
 5   Protein3            317 non-null    float64
 6   Protein4            317 non-null    float64
 7   Tumour_Stage        317 non-null    object 
 8   Histology           317 non-null    object 
 9   ER status           317 non-null    object 
 10  PR status           317 non-null    object 
 11  HER2 status         317 non-null    object 
 12  Surgery_type        317 non-null    object 
 13  Date_of_Surgery     317 non-null    object 
 14  Date_of_Last_Visit  317 non-null    object 
 15  Patient_Status      317 non-null    object 
dtypes: float

Breast cancer is mostly found in females,so let's look at the gender column to see how many females and males are there:

In [10]:
print(df.Gender.value_counts())

FEMALE    313
MALE        4
Name: Gender, dtype: int64


as expected ,the proprtion of females is more than males in the gender column.Now let's have a look at the stage of tumour of these patients:

In [12]:
stage = df["Tumour_Stage"].value_counts()
transactions = stage.index
quantity = stage.values 

fig = px.pie(df,
             values=quantity,
             names=transactions,hole = 0.5,
             title="Tumour Stages of Patients")
fig.show()

So most of the patients are in the secoand stage.Now let's have a look at the history of breast cancer patients.(Histology is a description of a tumor based on how abnormal the cancer cells and tissue look under a microscope and how quickly cancer can grow and spread):

In [13]:
histology = df["Histology"].value_counts()
transactions = histology.index
quantity = histology.values 
fig = px.pie(df,
             values=quantity,
             names=transactions,hole = 0.5,
             title="Histology of Patients")
fig.show()

according to above visualization Infiltrating Ductal Carcinoma is highest(70%).Now let's have a look at the values of ER status,PR status,and HER2 status of the patients:

In [15]:
print(df["ER status"].value_counts())
print(df["PR status"].value_counts())
print(df["HER2 status"].value_counts())

Positive    317
Name: ER status, dtype: int64
Positive    317
Name: PR status, dtype: int64
Negative    288
Positive     29
Name: HER2 status, dtype: int64


Now let's have a look at the type of surgeries done to the patients:

In [16]:
surgery = df["Surgery_type"].value_counts()
transactions = surgery.index
quantity = surgery.values
fig = px.pie(df,
             values=quantity,
             names=transactions,hole = 0.5,
             title = "Type of Surgery of Patients")
fig.show()

so,we explored the data ,the dataset has a lot of categorical features.To use this data to train a ML model ,we have to transform the values of all categorical columns.

In [17]:
from pandas.io.formats.info import DataFrameTableBuilder
df["Tumour_Stage"] = df["Tumour_Stage"].map({"I": 1, "II": 2, "III": 3})
df["Histology"] = df["Histology"].map({"Infiltrating Ductal Carcinoma": 1, 
                                           "Infiltrating Lobular Carcinoma": 2, "Mucinous Carcinoma": 3})
df["ER status"] = df["ER status"].map({"Positive": 1})
df["PR status"] = df["PR status"].map({"Positive": 1})
df["HER2 status"] = df["HER2 status"].map({"Positive": 1, "Negative": 2})
df["Gender"] = df["Gender"].map({"MALE": 0, "FEMALE": 1})
df["Surgery_type"] = df["Surgery_type"].map({"Other": 1, "Modified Radical Mastectomy": 2, 
                                                 "Lumpectomy": 3, "Simple Mastectomy": 4})
print(df.head())

     Patient_ID   Age  Gender  Protein1  Protein2  Protein3  Protein4  \
0  TCGA-D8-A1XD  36.0       1  0.080353   0.42638   0.54715  0.273680   
1  TCGA-EW-A1OX  43.0       1 -0.420320   0.57807   0.61447 -0.031505   
2  TCGA-A8-A079  69.0       1  0.213980   1.31140  -0.32747 -0.234260   
3  TCGA-D8-A1XR  56.0       1  0.345090  -0.21147  -0.19304  0.124270   
4  TCGA-BH-A0BF  56.0       1  0.221550   1.90680   0.52045 -0.311990   

   Tumour_Stage  Histology  ER status  PR status  HER2 status  Surgery_type  \
0             3          1          1          1            2             2   
1             2          3          1          1            2             3   
2             3          1          1          1            2             1   
3             2          1          1          1            2             2   
4             2          1          1          1            2             1   

  Date_of_Surgery Date_of_Last_Visit Patient_Status  
0       15-Jan-17          19-Ju

**BREAST CANCER SURVIVAL PREDICTION MODEL**

In [18]:
#splitting data
x = np.array(df[['Age', 'Gender', 'Protein1', 'Protein2', 'Protein3','Protein4', 
                   'Tumour_Stage', 'Histology', 'ER status', 'PR status', 
                   'HER2 status', 'Surgery_type']])
y = np.array(df[['Patient_Status']])
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.10, random_state=42)

In [20]:
model = SVC()
model.fit(xtrain, ytrain)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



SVC()

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
print(accuracy_score(ytest, model.predict(xtest)))

0.8125


now input the all features that we used to train this ML model and predict whether a patient will survive from breast cancer or not:

In [21]:
# features = [['Age', 'Gender', 'Protein1', 'Protein2', 'Protein3','Protein4', 'Tumour_Stage', 'Histology', 'ER status', 'PR status', 'HER2 status', 'Surgery_type']]
features = np.array([[36.0, 1, 0.080353, 0.42638, 0.54715, 0.273680, 3, 1, 1, 1, 2, 2,]])
print(model.predict(features))

['Alive']
